In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import os

In [2]:
'''
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH="/home/visit/learn_tf/ch5/model"
MODEL_NAME="mnist_model"
'''
BATCH_SIZE = 100 
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99 
MODEL_SAVE_PATH = "/home/visit/learn_tf/ch5/model"
MODEL_NAME = "mnist_model"

In [3]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name = "x-input")
    y_= tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name = "y-input")
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    
    global_step = tf.Variable(0, trainable = False)


    
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = y, 
        labels = tf.argmax(y_, 1))
    
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,
                                              global_step,
                                              mnist.train.num_examples / BATCH_SIZE ,
                                              LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y, labels = tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name = 'train')
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x : xs, y_ : ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (i, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step = global_step)
                
  

In [4]:
def main(argv=None):
    mnist = input_data.read_data_sets('/home/visit/learn_tf/ch5/mnist', one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()



Extracting /home/visit/learn_tf/ch5/mnist/train-images-idx3-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/train-labels-idx1-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step(s), loss on training batch is 2.86196.
After 1000 training step(s), loss on training batch is 0.23306.
After 2000 training step(s), loss on training batch is 0.156007.
After 3000 training step(s), loss on training batch is 0.161985.
After 4000 training step(s), loss on training batch is 0.127048.
After 5000 training step(s), loss on training batch is 0.106127.
After 6000 training step(s), loss on training batch is 0.10239.
After 7000 training step(s), loss on training batch is 0.0921367.
After 8000 training step(s), loss on training batch is 0.0805168.
After 9000 training step(s), loss on training batch is 0.0774883.
After 10000 training step(s), loss on training batch is 0.0686281.
After 11000 